In [24]:
import chromadb
import openai
import concurrent.futures
from tenacity import retry, stop_after_attempt, wait_random_exponential
import os
import dotenv

dotenv.load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")

In [55]:
client = chromadb.PersistentClient(path="/Users/jameslin/Documents/Code/Alexandria/apollo/chroma-db")
collection = client.get_collection(name="clinical_trials")

# client_asclepius = chromadb.PersistentClient(path="/Users/jameslin/Documents/Code/Alexandria/Asclepius/chroma-db")
# collection_asclepius = client_asclepius.get_collection(name="clinical_trials")


In [27]:
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def get_embedding(text: str, id: str, model="text-embedding-ada-002") -> tuple[int, list[float]]:
    try:
        result = openai.Embedding.create(input=[text], model=model)
    except:
        print("Error: " + str(id))
        print("Text: " + text)
        
    return result

In [59]:
query = "what is the best treatment for covid-19"
query_embedding = get_embedding(query, 'query')["data"][0]["embedding"]
collection.add(
    documents=[query],
    embeddings=[query_embedding],
    ids=["2"]
)
print(query_embedding)
collection.peek()
print(collection.get())
print(query_embedding)

answer = collection.query(
    query_embeddings=query_embedding,
    include=["documents"],
    n_results=1,
)
print(str(answer))

[-0.002094801515340805, 0.002597083570435643, 0.008392645046114922, -0.010287541896104813, -0.0016286368481814861, 0.011712074279785156, -0.012592327781021595, -0.021099204197525978, -0.005321837496012449, 0.015589220449328423, 0.008688302710652351, 0.0386505164206028, -0.015710171312093735, 0.026326969265937805, -0.01224291417747736, -0.01517261192202568, 0.047547124326229095, 0.000760142516810447, 0.013815274462103844, -0.012121963314712048, -0.020306304097175598, 0.026111945509910583, -0.0018176224548369646, -0.03437691926956177, -0.00622224947437644, 0.007599744945764542, 0.00985749438405037, -0.02459334023296833, -0.015199489891529083, -0.003977939020842314, -0.007794610224664211, -0.000789120327681303, -0.008607668802142143, 0.00011559625272639096, -0.012565449811518192, 0.00622224947437644, -0.0021032008808106184, -0.004028335213661194, 0.00886972900480032, -0.005150490440428257, 0.026434481143951416, -0.00738472118973732, -0.0012296669883653522, 0.006252487190067768, -0.0080029